# Worksheet 08

Name:  Priscilla Nguyen \
UID: U83582329

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[7.320157494336885, 5.430710809596048, 6.133942887685875, 5.7034649278969685, 4.4744046320886754, 5.757934567382505, 3.791123302116104, 5.784807269795199, 5.886276503189376, 5.795821222052611]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[6.754845478192443, 5.998221124131312, 9.409819382934486, 9.095763048508589, 9.009253415648704, 5.685441584035515, 7.078526012642615, 5.749671786421249, 8.673233251740616, 6.772248278151544]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[6.772248278151544, 5.795821222052611, 8.673233251740616, 5.886276503189376, 5.784807269795199, 3.791123302116104, 5.757934567382505, 4.4744046320886754, 5.749671786421249, 5.7034649278969685]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Means of the two Gaussian distributions: 5 and 8
- Variances of the two Gaussian distributions: 1 and 1
- Weights of the two distributions (which is uncertain due to the coin flip): Depends on the actual number of samples selected from each distribution.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1) / len(s1), sum(s2) / len(s2)]
var = [sum(map(lambda x: (x - mean[0])**2, s1)) / len(s1), sum(map(lambda x: (x - mean[1])**2, s2)) / len(s2)]


print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.772248278151544, 5.795821222052611, 5.886276503189376, 5.784807269795199, 3.791123302116104, 5.757934567382505, 4.4744046320886754, 5.749671786421249, 5.7034649278969685]
[8.673233251740616]
P(S_1) = 0.9,  P(S_2) = 0.1
mean_1 = 5.523972498788249,  mean_2 = 8.673233251740616
var_1 = 0.6748497904843037,  var_2 = 0.0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)


probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  6.772248278151544
probability of observing that point if it came from cluster 0 =  0.10684402563800761
probability of observing that point if it came from cluster 1 =  nan
point =  5.795821222052611
probability of observing that point if it came from cluster 0 =  0.5450875883835622
probability of observing that point if it came from cluster 1 =  nan
point =  8.673233251740616
probability of observing that point if it came from cluster 0 =  1.103655468569961e-05
probability of observing that point if it came from cluster 1 =  nan
point =  5.886276503189376
probability of observing that point if it came from cluster 0 =  0.5118180871110675
probability of observing that point if it came from cluster 1 =  nan
point =  5.784807269795199
probability of observing that point if it came from cluster 0 =  0.5486099499723837
probability of observing that point if it came from cluster 1 =  nan
point =  3.791123302116104
probability of observing that point if it came from cluster 0 =  0.02

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:1983: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:1983: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([(x - mean[0])**2 * prob for x, prob in zip(data, prob_s0_x)]) / sum(prob_s0_x), sum([(x - mean[1])**2 * prob for x, prob in zip(data, prob_s1_x)]) / sum(prob_s1_x)]


print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.9,  P(S_2) = 0.1
mean_1 = nan,  mean_2 = nan
var_1 = nan,  var_2 = nan


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_s0_x = []  # Updated P(S_0 | X_i)
prob_s1_x = []  # Updated P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        
    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Updated Probability of coming from S_1 = " + str(p[1]))
    print("Updated Probability of coming from S_2 = " + str(p[2]))
    print()

6.772248278151544
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

5.795821222052611
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

8.673233251740616
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

5.886276503189376
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

5.784807269795199
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

3.791123302116104
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

5.757934567382505
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

4.4744046320886754
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

5.749671786421249
Updated Probability of coming from S_1 = nan
Updated Probability of coming from S_2 = nan

5.7034649278969685

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
hard_assignments = [0 if prob0 > prob1 else 1 for prob0, prob1 in zip(prob_s0_x, prob_s1_x)]